In [1]:
import os
import pandas as pd
import numpy as np

# 导入PySpark
from pyspark import SparkContext
from pyspark.sql import SparkSession

# 导入pyspark 的部分函数
from pyspark.sql.functions import col, min, max, avg, lit

# 导入pyspark 的机器学习相关的一些包
from pyspark.ml.recommendation import ALS 
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator # Cross-Validation
from pyspark.ml.evaluation import RegressionEvaluator # Performance metric
import seaborn as sns
import matplotlib.pyplot as plt

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

pd.set_option('display.max_columns', 200)
pd.set_option('display.max_colwidth', 400)

In [2]:
import os
### bin文件所在的路径  C:\Program Files\Java\jdk-14.0.1\bin
os.environ['JAVA_HOME'] = 'C:\Program Files\Java\jdk-14.0.1'  # 这里的路径为java的bin目录所在路径

SparkContext是开发Spark应用的入口，它负责和整个集群的交互，包括创建RDD等。
从本质上来说，SparkContext是Spark的对外接口，负责向调用这提供Spark的各种功能。

In [3]:
from pyspark import SparkContext 
from pyspark.sql import SQLContext

In [4]:
sc = SparkContext(appName = "Book-Recommendation")
print(sc)

<SparkContext master=local[*] appName=Book-Recommendation>


#### 默认创建spark对话

In [5]:
spark = SparkSession.Builder().getOrCreate()

In [6]:
ratings = spark.read.csv('./input/ratings.csv',
                         header = True,
                         inferSchema=True)
print(type(ratings))

<class 'pyspark.sql.dataframe.DataFrame'>


In [7]:
ratings.show(4)

+------+-------+------+---------+
|userId|movieId|rating|timestamp|
+------+-------+------+---------+
|     1|      1|   4.0|964982703|
|     1|      3|   4.0|964981247|
|     1|      6|   4.0|964982224|
|     1|     47|   5.0|964983815|
+------+-------+------+---------+
only showing top 4 rows



In [10]:
## 删除timestamp列
ratings = ratings.drop('timestamp')
ratings.show(4)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
+------+-------+------+
only showing top 4 rows



In [12]:
## 显示头几行，默认显示1行
ratings.head(4)

[Row(userId=1, movieId=1, rating=4.0),
 Row(userId=1, movieId=3, rating=4.0),
 Row(userId=1, movieId=6, rating=4.0),
 Row(userId=1, movieId=47, rating=5.0)]

### 显示ratings的相关的信息

##### 查看数据结构

In [14]:
ratings.printSchema()

root
 |-- userId: integer (nullable = true)
 |-- movieId: integer (nullable = true)
 |-- rating: double (nullable = true)



##### 显示前几行

In [15]:
ratings.show(10)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
+------+-------+------+
only showing top 10 rows



In [18]:
## 限制数据的条数，并显示，显示大于实际行数，以实际行数为准
ratings.limit(10).show(11)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      3|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     50|   5.0|
|     1|     70|   3.0|
|     1|    101|   5.0|
|     1|    110|   4.0|
|     1|    151|   5.0|
|     1|    157|   5.0|
+------+-------+------+



In [19]:
### 选择数据，并显示(默认显示20行)
ratings.select('rating','userId','movieId').show()

+------+------+-------+
|rating|userId|movieId|
+------+------+-------+
|   4.0|     1|      1|
|   4.0|     1|      3|
|   4.0|     1|      6|
|   5.0|     1|     47|
|   5.0|     1|     50|
|   3.0|     1|     70|
|   5.0|     1|    101|
|   4.0|     1|    110|
|   5.0|     1|    151|
|   5.0|     1|    157|
|   5.0|     1|    163|
|   5.0|     1|    216|
|   3.0|     1|    223|
|   5.0|     1|    231|
|   4.0|     1|    235|
|   5.0|     1|    260|
|   3.0|     1|    296|
|   3.0|     1|    316|
|   5.0|     1|    333|
|   4.0|     1|    349|
+------+------+-------+
only showing top 20 rows



In [20]:
### 选择数据，并显示(默认显示20行)
ratings.select('rating','movieId').show(10)

+------+-------+
|rating|movieId|
+------+-------+
|   4.0|      1|
|   4.0|      3|
|   4.0|      6|
|   5.0|     47|
|   5.0|     50|
|   3.0|     70|
|   5.0|    101|
|   4.0|    110|
|   5.0|    151|
|   5.0|    157|
+------+-------+
only showing top 10 rows



#### 对数据做描述性统计分析

In [21]:
ratings.summary().show()

+-------+------------------+----------------+------------------+
|summary|            userId|         movieId|            rating|
+-------+------------------+----------------+------------------+
|  count|            100836|          100836|            100836|
|   mean|326.12756356856676|19435.2957177992| 3.501556983616962|
| stddev| 182.6184914635004|35530.9871987003|1.0425292390606342|
|    min|                 1|               1|               0.5|
|    25%|               177|            1199|               3.0|
|    50%|               325|            2991|               3.5|
|    75%|               477|            8092|               4.0|
|    max|               610|          193609|               5.0|
+-------+------------------+----------------+------------------+



In [22]:
ratings.select('rating').summary().show()

+-------+------------------+
|summary|            rating|
+-------+------------------+
|  count|            100836|
|   mean| 3.501556983616962|
| stddev|1.0425292390606342|
|    min|               0.5|
|    25%|               3.0|
|    50%|               3.5|
|    75%|               4.0|
|    max|               5.0|
+-------+------------------+



#### 查看数据的行数，列数

In [23]:
print('Number of rows: \t', ratings.count())
print('Number of columns: \t', len(ratings.columns))

Number of rows: 	 100836
Number of columns: 	 3


In [25]:
ratings.columns

['userId', 'movieId', 'rating']

#### 查看观众及电影的出现次数

In [27]:
ratings.groupBy('userId').count().show(10)

+------+-----+
|userId|count|
+------+-----+
|   148|   48|
|   463|   33|
|   471|   28|
|   496|   29|
|   243|   36|
|   392|   25|
|   540|   42|
|    31|   50|
|   516|   26|
|    85|   34|
+------+-----+
only showing top 10 rows



In [28]:
ratings.groupBy('movieId').count().show(10)

+-------+-----+
|movieId|count|
+-------+-----+
|   1580|  165|
|   2366|   25|
|   3175|   75|
|   1088|   42|
|  32460|    4|
|  44022|   23|
|  96488|    4|
|   1238|    9|
|   1342|   11|
|   1591|   26|
+-------+-----+
only showing top 10 rows



In [29]:
### 计算观众的平均评分
ratings.groupBy('userId').mean('rating').show(10)

+------+------------------+
|userId|       avg(rating)|
+------+------------------+
|   148|3.7395833333333335|
|   463| 3.787878787878788|
|   471|             3.875|
|   496| 3.413793103448276|
|   243| 4.138888888888889|
|   392|               3.2|
|   540|               4.0|
|    31|              3.92|
|   516|3.6923076923076925|
|    85|3.7058823529411766|
+------+------------------+
only showing top 10 rows



In [30]:
### 计算电影的平均评分
ratings.groupBy('movieId').mean('rating').show(10)

+-------+------------------+
|movieId|       avg(rating)|
+-------+------------------+
|   1580| 3.487878787878788|
|   2366|              3.64|
|   3175|              3.58|
|   1088| 3.369047619047619|
|  32460|              4.25|
|  44022| 3.217391304347826|
|  96488|              4.25|
|   1238| 4.055555555555555|
|   1342|               2.5|
|   1591|2.6346153846153846|
+-------+------------------+
only showing top 10 rows



In [32]:
### 统计是否每列的缺失值数量
for col in ratings.columns:
    # 对某列按照条件过滤
    print(col.ljust(10), ratings.filter(ratings[col].isNull()).count())

userId     0
movieId    0
rating     0


In [34]:
### 计算观众和电影交互二维表的评分缺失率
numerator = ratings.select("rating").count()

num_users = ratings.select("userId").distinct().count()
num_items = ratings.select("movieId").distinct().count()

denominator = num_users * num_items

sparsity = (1.0 - (numerator * 1.0)/ denominator) * 100
print("The ratings dataframe is ", "%.2f" % sparsity + "% empty.")

The ratings dataframe is  98.30% empty.


In [35]:
print('观众数量：',num_users)
print('电影数量：',num_items)

观众数量： 610
电影数量： 9724


In [48]:
print("Item with the fewest ratings: ")
ratings.groupBy("movieId").count().select(min("count")).show()

Item with the fewest ratings: 
+----------+
|min(count)|
+----------+
|         1|
+----------+



In [49]:
print("Avg num ratings per item: ")
ratings.groupBy("movieid").count().select(avg("count")).show()  #不区分大小写

Avg num ratings per item: 
+------------------+
|        avg(count)|
+------------------+
|10.369806663924312|
+------------------+



In [50]:
# Avg num ratings per users
print("Avg num ratings per user: ")
ratings.groupBy("userid").count().select(avg("count")).show()

Avg num ratings per user: 
+------------------+
|        avg(count)|
+------------------+
|165.30491803278687|
+------------------+



In [35]:
#baseline 

In [51]:
als = ALS(userCol="userId", itemCol="movieId", ratingCol="rating", 
          nonnegative = True, # Non negative matrix factorization
          coldStartStrategy = "drop", # What to do if user do not appear in train and test set
          implicitPrefs = False) # Explicit preference

In [52]:
type(als)

pyspark.ml.recommendation.ALS

In [53]:
param_grid = ParamGridBuilder() \
            .addGrid(als.rank, [10, 50, 100]) \
            .addGrid(als.maxIter, [5, 50, 100]) \
            .addGrid(als.regParam, [.01, .05, .1]) \
            .build()

In [54]:
# Define evaluator as RMSE
evaluator = RegressionEvaluator(metricName = "rmse", 
                                labelCol = "rating", 
                                predictionCol = "prediction")
# Print length of evaluator
print ("Num models to be tested: ", len(param_grid))

Num models to be tested:  27


In [55]:
cv = CrossValidator(estimator = als, 
                    estimatorParamMaps = param_grid, 
                    evaluator = evaluator, 
                    numFolds = 5)

In [56]:
# Create test and train set
(train, test) = ratings.randomSplit([0.8, 0.2], 
                                    seed = 1234)
print(type(train))

<class 'pyspark.sql.dataframe.DataFrame'>


In [57]:
train.show(4)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      1|   4.0|
|     1|      6|   4.0|
|     1|     47|   5.0|
|     1|     70|   3.0|
+------+-------+------+
only showing top 4 rows



In [58]:
test.show(4)

+------+-------+------+
|userId|movieId|rating|
+------+-------+------+
|     1|      3|   4.0|
|     1|     50|   5.0|
|     1|    362|   5.0|
|     1|    441|   4.0|
+------+-------+------+
only showing top 4 rows



In [59]:
als_mod = als.fit(train)

In [60]:
test_pred = als_mod.transform(test)

In [61]:
test_pred.show(n = 10)

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   597|    471|   2.0| 4.5308647|
|   436|    471|   3.0| 3.8087804|
|   218|    471|   4.0| 2.9790125|
|   387|    471|   3.0|  3.039295|
|   217|    471|   2.0| 3.0220342|
|   287|    471|   4.5| 2.1219225|
|    32|    471|   3.0| 3.9152074|
|   260|    471|   4.5| 3.1810186|
|   104|    471|   4.5| 3.8068326|
|   111|   1088|   3.0| 3.3439498|
+------+-------+------+----------+
only showing top 10 rows



In [62]:
print(evaluator.evaluate(test_pred))

0.8794302256746631


In [63]:
best_model = als_mod

In [64]:
test_predictions = best_model.transform(test)

In [65]:
# View the predictions 
test_predictions.show(n = 10)

# Calculate and print the RMSE of test_predictions
print ("RMSE: ", evaluator.evaluate(test_predictions))

+------+-------+------+----------+
|userId|movieId|rating|prediction|
+------+-------+------+----------+
|   597|    471|   2.0| 4.5308647|
|   436|    471|   3.0| 3.8087804|
|   218|    471|   4.0| 2.9790125|
|   387|    471|   3.0|  3.039295|
|   217|    471|   2.0| 3.0220342|
|   287|    471|   4.5| 2.1219225|
|    32|    471|   3.0| 3.9152074|
|   260|    471|   4.5| 3.1810186|
|   104|    471|   4.5| 3.8068326|
|   111|   1088|   3.0| 3.3439498|
+------+-------+------+----------+
only showing top 10 rows

RMSE:  0.8794302256746631


In [66]:
# Generate n recommendations for all users
ALS_recommendations = best_model.recommendForAllUsers(numItems = 10) 

In [67]:
ALS_recommendations.show(n = 10)

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   471|[[86781, 4.63092]...|
|   463|[[3379, 5.0837936...|
|   496|[[8477, 5.5474496...|
|   148|[[3022, 4.5687995...|
|   540|[[3379, 5.489201]...|
|   392|[[7982, 5.6097746...|
|   243|[[72171, 6.774387...|
|    31|[[3925, 5.5229645...|
|   516|[[4429, 4.8815417...|
|   580|[[3837, 5.282003]...|
+------+--------------------+
only showing top 10 rows



In [68]:
# Temporary table
ALS_recommendations.registerTempTable("ALS_recs_temp")

In [71]:
clean_recs = spark.sql("""SELECT userid,
                            movieIds_and_ratings.movieid AS movieid,
                            movieIds_and_ratings.rating AS prediction
                        FROM ALS_recs_temp
                        LATERAL VIEW explode(recommendations) exploded_table
                            AS movieIds_and_ratings""")
clean_recs.show()

+------+-------+----------+
|userid|movieid|prediction|
+------+-------+----------+
|   471|  86781|   4.63092|
|   471|   7767|  4.609346|
|   471|    599| 4.5863476|
|   471|  78836|  4.577664|
|   471|  89904| 4.5733204|
|   471|  26810| 4.5718446|
|   471|  49347|  4.566446|
|   471|  33649|  4.555927|
|   471| 117531| 4.5537324|
|   471|   7071| 4.5537324|
|   463|   3379| 5.0837936|
|   463| 171495|   4.94011|
|   463|  33649| 4.9318395|
|   463|  60943| 4.9245157|
|   463|  59018| 4.9245157|
|   463|  78836| 4.9089823|
|   463| 184245| 4.9006357|
|   463| 179135| 4.9006357|
|   463|   7071| 4.9006357|
|   463|  26073| 4.9006357|
+------+-------+----------+
only showing top 20 rows



In [72]:
# Recommendations for unread books
(clean_recs.join(ratings, ["userid", "movieid"], "left")
    .filter(ratings.rating.isNull()).show())

+------+-------+----------+------+
|userid|movieid|prediction|rating|
+------+-------+----------+------+
|   471|  86781|   4.63092|  null|
|   471|   7767|  4.609346|  null|
|   471|    599| 4.5863476|  null|
|   471|  78836|  4.577664|  null|
|   471|  89904| 4.5733204|  null|
|   471|  26810| 4.5718446|  null|
|   471|  49347|  4.566446|  null|
|   471|  33649|  4.555927|  null|
|   471| 117531| 4.5537324|  null|
|   471|   7071| 4.5537324|  null|
|   463|   3379| 5.0837936|  null|
|   463| 171495|   4.94011|  null|
|   463|  33649| 4.9318395|  null|
|   463|  60943| 4.9245157|  null|
|   463|  59018| 4.9245157|  null|
|   463|  78836| 4.9089823|  null|
|   463| 184245| 4.9006357|  null|
|   463| 179135| 4.9006357|  null|
|   463|   7071| 4.9006357|  null|
|   463|  26073| 4.9006357|  null|
+------+-------+----------+------+
only showing top 20 rows



In [73]:
new_books = (clean_recs.join(ratings, ["userid", "movieid"], "left")
    .filter(ratings.rating.isNull()))

In [74]:
new_books.show()

+------+-------+----------+------+
|userid|movieid|prediction|rating|
+------+-------+----------+------+
|   471|  86781|   4.63092|  null|
|   471|   7767|  4.609346|  null|
|   471|    599| 4.5863476|  null|
|   471|  78836|  4.577664|  null|
|   471|  89904| 4.5733204|  null|
|   471|  26810| 4.5718446|  null|
|   471|  49347|  4.566446|  null|
|   471|  33649|  4.555927|  null|
|   471| 117531| 4.5537324|  null|
|   471|   7071| 4.5537324|  null|
|   463|   3379| 5.0837936|  null|
|   463| 171495|   4.94011|  null|
|   463|  33649| 4.9318395|  null|
|   463|  60943| 4.9245157|  null|
|   463|  59018| 4.9245157|  null|
|   463|  78836| 4.9089823|  null|
|   463| 184245| 4.9006357|  null|
|   463| 179135| 4.9006357|  null|
|   463|   7071| 4.9006357|  null|
|   463|  26073| 4.9006357|  null|
+------+-------+----------+------+
only showing top 20 rows



In [75]:
print(new_books.count())

5821
